In [1]:
# Run this cell to update data after doing a git pull on the COVID-19 subdirectory

import csv
import pandas as pd
import altair as alt

# show charts inline
alt.renderers.enable('default') #note: you may have to change 'default' to 'notebook' depending on your configuration

taiwan_population = 23780000
singapore_population = 5639000
hongkong_population = 7451000
china_population = 1427647786  # mainland
usa_population = 329515103
italy_population = 60360000
southkorea_population = 51640000
japan_population = 126500000


# Convert a list of strings into a list of ints
def to_int(row: list) -> list:
    return [int(r) for r in row]


# Add values corresponding to the same position in multiple lists 
# (for countries that have multiple regions in the data)
def add_row(old_row: list, new_row: list) -> list:
    if old_row == []:
        return to_int(new_row)
    else:
        return[o + int(n) for o, n in zip(old_row, new_row)]
            

# Get confirmed cases
china_confirmed = []
with open('./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[1] == "Country/Region":
            dates = pd.to_datetime(row[4:], format="%m/%d/%y")
        elif row[1] == "Taiwan*":
            taiwan_confirmed = to_int(row[4:])
        elif row[1] == "Singapore":
            singapore_confirmed = to_int(row[4:])
        elif row[0] == "Hong Kong":
            hongkong_confirmed = to_int(row[4:])
        elif row[1] == "China" and row[0] != "Hong Kong":  # mainland
            china_confirmed = add_row(china_confirmed, row[4:])
        elif row[1] == "US":  
            usa_confirmed = to_int(row[4:])
        elif row[1] == "Italy":  
            italy_confirmed = to_int(row[4:])
        elif row[1] == "Korea, South":  
            southkorea_confirmed = to_int(row[4:])
        elif row[1] == "Japan":  
            japan_confirmed = to_int(row[4:])
        
            
            
# Get deaths
china_deaths = []
with open('./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[1] == "Taiwan*":
            taiwan_deaths = to_int(row[4:])
        elif row[1] == "Singapore":
            singapore_deaths = to_int(row[4:])
        elif row[0] == "Hong Kong":
            hongkong_deaths = to_int(row[4:])
        elif row[1] == "China" and row[0] != "Hong Kong":  # mainland
            china_deaths = add_row(china_deaths, row[4:])
        elif row[1] == "US": 
            usa_deaths = to_int(row[4:])
        elif row[1] == "Italy": 
            italy_deaths = to_int(row[4:])
        elif row[1] == "Korea, South":  
            southkorea_deaths = to_int(row[4:])
        elif row[1] == "Japan":  
            japan_deaths = to_int(row[4:])
            
            
            
as_of_date = dates[-1] # The last date that was recorded
            

In [2]:
# Run this cell to get current stats for the countries we care about

# Confirmed over population
taiwan_cop = taiwan_confirmed[-1]/taiwan_population
singapore_cop = singapore_confirmed[-1]/singapore_population
hongkong_cop = hongkong_confirmed[-1]/hongkong_population
china_cop = china_confirmed[-1]/china_population
usa_cop = usa_confirmed[-1]/usa_population
italy_cop = italy_confirmed[-1]/italy_population
southkorea_cop = southkorea_confirmed[-1]/southkorea_population
japan_cop = japan_confirmed[-1]/japan_population

# Deaths over confirmed
taiwan_doc = taiwan_deaths[-1]/taiwan_confirmed[-1]
singapore_doc = singapore_deaths[-1]/singapore_confirmed[-1]
hongkong_doc = hongkong_deaths[-1]/hongkong_confirmed[-1]
china_doc = china_deaths[-1]/china_confirmed[-1]
usa_doc = usa_deaths[-1]/usa_confirmed[-1]
italy_doc = italy_deaths[-1]/italy_confirmed[-1]
southkorea_doc = southkorea_deaths[-1]/southkorea_confirmed[-1]
japan_doc = japan_deaths[-1]/japan_confirmed[-1]

# Deaths over population
taiwan_dop = taiwan_deaths[-1]/taiwan_population
singapore_dop = singapore_deaths[-1]/singapore_population
hongkong_dop = hongkong_deaths[-1]/hongkong_population
china_dop = china_deaths[-1]/china_population
usa_dop = usa_deaths[-1]/usa_population
italy_dop = italy_deaths[-1]/italy_population
southkorea_dop = southkorea_deaths[-1]/southkorea_population
japan_dop = japan_deaths[-1]/japan_population

print('As of', as_of_date.strftime("%Y-%m-%d"))
print('Region, Confirmed, Deaths')
print('Taiwan', taiwan_confirmed[-1], taiwan_deaths[-1])
print('Singapore', singapore_confirmed[-1], singapore_deaths[-1])
print('Hong Kong', hongkong_confirmed[-1], hongkong_deaths[-1])
print('South Korea', southkorea_confirmed[-1], southkorea_deaths[-1])
print('Japan', japan_confirmed[-1], japan_deaths[-1])
print('China', china_confirmed[-1], china_deaths[-1])
print('USA', usa_confirmed[-1], usa_deaths[-1])
print('Italy', italy_confirmed[-1], italy_deaths[-1])
print('Total HK, Taiwan, Singapore', hongkong_confirmed[-1]+singapore_confirmed[-1]+taiwan_confirmed[-1], hongkong_deaths[-1]+singapore_deaths[-1]+taiwan_deaths[-1])

As of 2020-04-14
Region, Confirmed, Deaths
Taiwan 393 6
Singapore 3252 10
Hong Kong 1012 4
South Korea 10564 222
Japan 7645 143
China 82294 3341
USA 607670 25787
Italy 162488 21067
Total HK, Taiwan, Singapore 4657 20


In [12]:
# Run the following cells to get charts comparing rates of infection, death, and population

regions = ['Taiwan', 'Singapore', 'Hong Kong', 'South Korea', 'Japan', ' China (Mainland)', 'USA', 'Italy']

# Total Cases
confirmed = [taiwan_confirmed[-1], singapore_confirmed[-1], hongkong_confirmed[-1], southkorea_confirmed[-1], japan_confirmed[-1], china_confirmed[-1], usa_confirmed[-1], italy_confirmed[-1]]
data = pd.DataFrame(list(zip(regions, confirmed)), columns=['Region', 'Cases'])

bars = alt.Chart(data).mark_bar().encode(
    x="Cases:Q",
    y=alt.Y("Region:O", sort='x')
).properties(
    title={
        "text":'COVID-19: Confirmed Cases',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("Cases:Q", format=",")
    )
)

alt.layer(bars, text).configure_axisX(
    labelAngle=-45
).configure_axisY(
    title=None
)

alt.LayerChart(...)

In [4]:
# Total Deaths
deaths = [taiwan_deaths[-1], singapore_deaths[-1], hongkong_deaths[-1], southkorea_deaths[-1], japan_deaths[-1], china_deaths[-1], usa_deaths[-1], italy_deaths[-1]]

data = pd.DataFrame(list(zip(regions, deaths)), columns=['Region', 'Deaths'])

bars = alt.Chart(data).mark_bar().encode(
    x="Deaths:Q",
    y=alt.Y("Region:O", sort='x')
).properties(
    title={
        "text":'COVID-19: Deaths',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("Deaths:Q", format=",")
    )
)

alt.layer(bars, text).configure_axisX(
    labelAngle=-45
).configure_axisY(
    title=None
).configure_bar(
    color='darkred'
)

alt.LayerChart(...)

In [5]:
# Confirmed and Deaths, Small Regions
data = pd.DataFrame([['Taiwan', taiwan_confirmed[-1], 'Confirmed'],
                    ['Taiwan', taiwan_deaths[-1], 'Deaths'],
                    ['Singapore', singapore_confirmed[-1], 'Confirmed'],
                    ['Singapore', singapore_deaths[-1], 'Deaths'],
                    ['Hong Kong', hongkong_confirmed[-1], 'Confirmed'],
                    ['Hong Kong', hongkong_deaths[-1], 'Deaths'],
                    ['South Korea', southkorea_confirmed[-1], 'Confirmed'],
                    ['South Korea', southkorea_deaths[-1], 'Deaths'],
                    ['Japan', japan_confirmed[-1], 'Confirmed'],
                    ['Japan', japan_deaths[-1], 'Deaths']], 
                    columns=['Region', 'Count', 'Case Type'])

bars = alt.Chart(data).mark_bar().encode(
    x="Count:Q",
    y="Case Type:N",
    color=alt.Color("Case Type:N", legend=None, scale=alt.Scale(range=['steelblue','darkred']))
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("Count:Q", format=",")
    )
)

alt.layer(bars, text).facet(
    row="Region:N"
).configure_axisX(
    labelAngle=-45,
    title=None
).configure_axisY(
    title=None
).properties(
    title={
        "text":'COVID-19: Regions with Few Confirmed Cases',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

alt.FacetChart(...)

In [6]:
# Confirmed and Deaths, Large Regions
regions_small = ['Taiwan', 'Singapore', 'Hong Kong', 'South Korea', 'Japan']
confirmed_small = [taiwan_confirmed[-1], singapore_confirmed[-1], hongkong_confirmed[-1], southkorea_confirmed[-1], japan_confirmed[-1]]
deaths_small = [taiwan_deaths[-1], singapore_deaths[-1], hongkong_deaths[-1], southkorea_deaths[-1], japan_deaths[-1]]

data = pd.DataFrame([['China (Mainland)', china_confirmed[-1], 'Confirmed'],
                    ['China (Mainland)', china_deaths[-1], 'Deaths'],
                    ['USA', usa_confirmed[-1], 'Confirmed'],
                    ['USA', usa_deaths[-1], 'Deaths'],
                    ['Italy', italy_confirmed[-1], 'Confirmed'],
                    ['Italy', italy_deaths[-1], 'Deaths']], 
                    columns=['Region', 'Count', 'Case Type'])

bars = alt.Chart(data).mark_bar().encode(
    x="Count:Q",
    y="Case Type:N",
    color=alt.Color("Case Type:N", legend=None, scale=alt.Scale(range=['steelblue','darkred']))
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("Count:Q", format=",")
    )
)

alt.layer(bars, text).facet(
    row="Region:N"
).configure_axisX(
    labelAngle=-45,
    title=None
).configure_axisY(
    title=None
).properties(
    title={
        "text":'COVID-19: Regions with Many Confirmed Cases',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

alt.FacetChart(...)

In [7]:
# Confirmed over population
cop = [taiwan_cop, singapore_cop, hongkong_cop, southkorea_cop, japan_cop, china_cop, usa_cop, italy_cop]

data = pd.DataFrame(list(zip(regions, cop)), columns=['Region', '%'])

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X("%:Q", axis=alt.Axis(format='%')),
    y=alt.Y("Region:O", sort='x')
).properties(
    title={
        "text":'COVID-19: Confirmed Cases as % of Population',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("%:Q", format=".3%")
    )
)

alt.layer(bars, text).configure_axisX(
    labelAngle=-45,
    title=None
).configure_axisY(
    title=None
)

alt.LayerChart(...)

In [8]:
# Deaths over confirmed
doc = [taiwan_doc, singapore_doc, hongkong_doc, southkorea_doc, japan_doc, china_doc, usa_doc, italy_doc]

data = pd.DataFrame(list(zip(regions, doc)), columns=['Region', '%'])

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X("%:Q", axis=alt.Axis(format='%')),
    y=alt.Y("Region:O", sort='x')
).properties(
    title={
        "text":'COVID-19: Deaths as % of Confirmed Cases',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("%:Q", format=".2%")
    )
)

alt.layer(bars, text).configure_axisX(
    labelAngle=0,
    title=None
).configure_axisY(
    title=None
).configure_bar(
    color='darkred'
)

alt.LayerChart(...)

In [9]:
# Deaths over population
dop = [taiwan_dop, singapore_dop, hongkong_dop, southkorea_dop, japan_dop, china_dop, usa_dop, italy_dop]

data = pd.DataFrame(list(zip(regions, dop)), columns=['Region', '%'])

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X("%:Q", axis=alt.Axis(format='%')),
    y=alt.Y("Region:O", sort='x')
).properties(
    title={
        "text":'COVID-19: Deaths as % of Population',
        "subtitle":'As of ' + as_of_date.strftime("%B %d, %Y")
    }
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=(
        alt.Text("%:Q", format=".5%")
    )
)

alt.layer(bars, text).configure_axisX(
    labelAngle=-45,
    title=None
).configure_axisY(
    title=None
).configure_bar(
    color='darkred'
)

alt.LayerChart(...)

In [11]:
# Trajectory

#todo
# plt.plot(dates, usa_confirmed)
# plt.title("USA Confirmed Cases")